In [8]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

# Schools Data

In [9]:
schools_data = pd.read_csv("schools_clean.csv")
schools_data.head()

,Publication Name,Abbreviated Name,School Level,Admission Type,Current Grade Span Served,GPS Location,Street Address,Website,SPR_Score
0,John Bartram High School,Bartram,High,Neighborhood,'09-12,"39.92149737, -75.23405562",2401 S 67TH ST,http://philasd.org/bartram,19
1,West Philadelphia High School,West Philadelphia,High,Neighborhood,'09-12,"39.958110, -75.219654",4901 CHESTNUT ST,http://philasd.org/wphs,23
2,High School of the Future,HS of Future,High,Citywide,'09-12,"39.97465766, -75.2047566",4021 PARKSIDE AVE,http://philasd.org/sof,24
3,Paul Robeson High School for Human Services,Robeson,High,Citywide,'09-12,"39.95679553, -75.20476311",4125 LUDLOW ST,http://philasd.org/robeson,49
4,William L. Sayre High School,Sayre,High,Neighborhood,'09-12,"39.95759354, -75.23859405",5800 WALNUT ST,http://philasd.org/sayre,12


In [10]:
schools_data.rename(columns = {"Publication Name" : "school_name", 
                               "Abbreviated Name": "abbrv_name",
                               "School Level": "school_level",
                               "Admission Type": "type",
                               "Current Grade Span Served": "grade_level",
                               "GPS Location":"geolocation",
                               "Street Address": "address",
                               "Website": "website",
                               "SPR_Score": "spr_score"
                              }, inplace=True)

In [11]:
schools_data.index.name = 'id'
schools_data.head()

,school_name,abbrv_name,school_level,type,grade_level,geolocation,address,website,spr_score
id,,,,,,,,,
0,John Bartram High School,Bartram,High,Neighborhood,'09-12,"39.92149737, -75.23405562",2401 S 67TH ST,http://philasd.org/bartram,19
1,West Philadelphia High School,West Philadelphia,High,Neighborhood,'09-12,"39.958110, -75.219654",4901 CHESTNUT ST,http://philasd.org/wphs,23
2,High School of the Future,HS of Future,High,Citywide,'09-12,"39.97465766, -75.2047566",4021 PARKSIDE AVE,http://philasd.org/sof,24
3,Paul Robeson High School for Human Services,Robeson,High,Citywide,'09-12,"39.95679553, -75.20476311",4125 LUDLOW ST,http://philasd.org/robeson,49
4,William L. Sayre High School,Sayre,High,Neighborhood,'09-12,"39.95759354, -75.23859405",5800 WALNUT ST,http://philasd.org/sayre,12


## Transfering data to SQLite Database

In [12]:
class Schools(Base):
    __tablename__ = 'schools'
    id = Column(Integer, primary_key=True)
    school_name = Column(String)
    abbrv_name = Column(String)
    school_level = Column(String)
    type = Column(String)
    grade_level = Column(String)
    geolocation = Column(String)
    address= Column(String)
    website = Column(String)
    spr_score = Column(Integer)

In [13]:
Base.metadata.tables

immutabledict({'schools': Table('schools', MetaData(bind=None), Column('id', Integer(), table=<schools>, primary_key=True, nullable=False), Column('school_name', String(), table=<schools>), Column('abbrv_name', String(), table=<schools>), Column('school_level', String(), table=<schools>), Column('type', String(), table=<schools>), Column('grade_level', String(), table=<schools>), Column('geolocation', String(), table=<schools>), Column('address', String(), table=<schools>), Column('website', String(), table=<schools>), Column('spr_score', Integer(), table=<schools>), schema=None)})

In [14]:
engine = create_engine('sqlite:///philly.sqlite', echo=True)
Base.metadata.create_all(engine)

2021-05-08 09:23:05,648 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-05-08 09:23:05,649 INFO sqlalchemy.engine.base.Engine ()
2021-05-08 09:23:05,650 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-05-08 09:23:05,651 INFO sqlalchemy.engine.base.Engine ()
2021-05-08 09:23:05,652 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("schools")
2021-05-08 09:23:05,653 INFO sqlalchemy.engine.base.Engine ()


In [15]:
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [16]:
session.execute("SELECT * FROM schools;").fetchall()

2021-05-08 09:23:07,026 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-05-08 09:23:07,026 INFO sqlalchemy.engine.base.Engine SELECT * FROM schools;
2021-05-08 09:23:07,027 INFO sqlalchemy.engine.base.Engine ()


[(0, 'John Bartram High School', 'Bartram', 'High', 'Neighborhood', "'09-12", '39.92149737, -75.23405562', '2401 S 67TH ST', 'http://philasd.org/bartram'),
 (1, 'West Philadelphia High School', 'West Philadelphia', 'High', 'Neighborhood', "'09-12", '39.958110, -75.219654', '4901 CHESTNUT ST', 'http://philasd.org/wphs'),
 (2, 'High School of the Future', 'HS of Future', 'High', 'Citywide', "'09-12", '39.97465766, -75.2047566', '4021 PARKSIDE AVE', 'http://philasd.org/sof'),
 (3, 'Paul Robeson High School for Human Services', 'Robeson', 'High', 'Citywide', "'09-12", '39.95679553, -75.20476311', '4125 LUDLOW ST', 'http://philasd.org/robeson'),
 (4, 'William L. Sayre High School', 'Sayre', 'High', 'Neighborhood', "'09-12", '39.95759354, -75.23859405', '5800 WALNUT ST', 'http://philasd.org/sayre'),
 (5, 'William T. Tilden School', 'Tilden', 'Middle', 'Neighborhood', "'05-08", '39.92093619, -75.23227632', '6601 ELMWOOD AVE', 'http://philasd.org/tilden'),
 (6, 'Motivation High School', 'Motiv

In [17]:
session.execute('DROP TABLE "schools"')

2021-05-08 09:23:10,025 INFO sqlalchemy.engine.base.Engine DROP TABLE "schools"
2021-05-08 09:23:10,026 INFO sqlalchemy.engine.base.Engine ()


In [18]:
Base.metadata.tables["schools"].create(bind=engine)

2021-05-08 09:23:10,733 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE schools (
	id INTEGER NOT NULL, 
	school_name VARCHAR, 
	abbrv_name VARCHAR, 
	school_level VARCHAR, 
	type VARCHAR, 
	grade_level VARCHAR, 
	geolocation VARCHAR, 
	address VARCHAR, 
	website VARCHAR, 
	spr_score INTEGER, 
	PRIMARY KEY (id)
)


2021-05-08 09:23:10,734 INFO sqlalchemy.engine.base.Engine ()
2021-05-08 09:23:10,736 INFO sqlalchemy.engine.base.Engine COMMIT


In [19]:
sqlite_table = "schools"
schools_data.to_sql(sqlite_table, engine, if_exists = "append", index =True)

2021-05-08 09:23:11,451 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("schools")
2021-05-08 09:23:11,452 INFO sqlalchemy.engine.base.Engine ()
2021-05-08 09:23:11,455 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-05-08 09:23:11,459 INFO sqlalchemy.engine.base.Engine INSERT INTO schools (id, school_name, abbrv_name, school_level, type, grade_level, geolocation, address, website, spr_score) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2021-05-08 09:23:11,459 INFO sqlalchemy.engine.base.Engine ((0, 'John Bartram High School', 'Bartram', 'High', 'Neighborhood', "'09-12", '39.92149737, -75.23405562', '2401 S 67TH ST', 'http://philasd.org/bartram', '19'), (1, 'West Philadelphia High School', 'West Philadelphia', 'High', 'Neighborhood', "'09-12", '39.958110, -75.219654', '4901 CHESTNUT ST', 'http://philasd.org/wphs', '23'), (2, 'High School of the Future', 'HS of Future', 'High', 'Citywide', "'09-12", '39.97465766, -75.2047566', '4021 PARKSIDE AVE', 'http://philasd.org/sof', '

In [20]:
#session.execute('alter table schools drop id')

In [21]:
engine.table_names()
#session.execute('alter table schools add primary key(id)')

2021-05-08 09:23:12,774 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-05-08 09:23:12,775 INFO sqlalchemy.engine.base.Engine ()


['census_household',
 'censustracts',
 'household',
 'philly_schools',
 'query_url',
 'schools']

In [22]:
session.close()

2021-05-08 09:23:13,461 INFO sqlalchemy.engine.base.Engine ROLLBACK
